add intuitive charting / editable charts / timeframes 
fix to be a supportive trade signal, or a working strategy 

In [28]:
from datetime import date

date1 = date(2024, 1, 21)  
date2 = date(2025, 7, 3) 

# Calculate the difference
delta = date2 - date1

# Access the number of days
days_difference = delta.days

print(f"Number of days between {date1} and {date2}: {days_difference}")

Number of days between 2024-01-21 and 2025-07-03: 529


In [ ]:
## Copy fidelity option chain data to clipboard, set settings and run all 
ticker = "TSLA"
expiry = "2025-07-18"
dateexpiry = expiry
days = 240
exchange = "NASDAQ"
strike = 310
type = "p"
interval = 1 # in minutes "D" daily or "M" monthly 
filenum = ""
import yfinance as yf
import pandas as pd
pd.set_option('display.max_rows', None)  # Show all rows

## Fetching options chain for a specific ticker



In [30]:
import websocket
import json
import time
import threading
import uuid
import pandas as pd
from datetime import datetime
import logging

class TradingViewDataFetcher:
    def __init__(self, log_level=logging.WARNING):
        """
        Initialize the TradingView data fetcher with configurable logging.
        
        Args:
            log_level: Logging level (default: WARNING to suppress info messages)
        """
        logging.basicConfig(level=log_level, format='%(asctime)s - %(levelname)s - %(message)s')
        self.logger = logging.getLogger(__name__)
        
        # Default WebSocket URL (can be changed if needed)
        self.ws_url = "wss://prodata.tradingview.com/socket.io/websocket"
        
        # Default headers
        self.headers = {
            "Origin": "https://www.tradingview.com",
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
        }

    def fetch_data(self, symbol, exchange="NASDAQ", interval="D", num_bars=100, timeout=20, alt_urls=None):
        """
        Fetch historical price data from TradingView.
        
        Args:
            symbol: Trading symbol (e.g., "AAPL")
            exchange: Exchange name (e.g., "NASDAQ")
            interval: Time interval (e.g., "D" for daily, "W" for weekly, "M" for monthly, 
                     or "1" for 1 minute, "5" for 5 minutes, etc.)
            num_bars: Number of bars to request
            timeout: Maximum time to wait for data in seconds
            alt_urls: List of alternative WebSocket URLs to try if the primary fails
            
        Returns:
            pandas.DataFrame: DataFrame containing OHLCV data
        """
        urls_to_try = [self.ws_url]
        if alt_urls:
            urls_to_try.extend(alt_urls)
        
        for url in urls_to_try:
            self.logger.info(f"Trying WebSocket URL: {url}")
            
            # Reset state for each attempt
            self.session_id = f"qs_{uuid.uuid4().hex[:12]}"
            self.chart_session = f"cs_{uuid.uuid4().hex[:12]}"
            self.received_data = []
            self.error_occurred = False
            
            # Create and start WebSocket
            websocket.enableTrace(False)  # Set to True for verbose WebSocket logs
            ws = websocket.WebSocketApp(
                url,
                header=[f"{k}: {v}" for k, v in self.headers.items()],
                on_open=self._on_open(symbol, exchange, interval, num_bars),
                on_message=self._on_message,
                on_error=self._on_error,
                on_close=self._on_close
            )
            
            wst = threading.Thread(target=ws.run_forever)
            wst.daemon = True
            wst.start()
            
            try:
                # Wait for data
                start_time = time.time()
                while time.time() - start_time < timeout:
                    if self.error_occurred:
                        self.logger.error("Connection error detected, trying next URL if available")
                        break
                    
                    if len(self.received_data) >= num_bars/2:
                        self.logger.info(f"Received sufficient data: {len(self.received_data)} bars")
                        break
                        
                    time.sleep(0.5)
                
                # If we have data, process and return it
                if self.received_data:
                    ws.close()
                    return self._process_data()
                
            except KeyboardInterrupt:
                self.logger.info("Interrupted by user")
            finally:
                ws.close()
        
        # If we get here, all URLs failed
        self.logger.error("Failed to retrieve data from all WebSocket URLs")
        return None

    def _process_data(self):
        """Process the received data into a DataFrame."""
        df = pd.DataFrame(self.received_data)
        if not df.empty:
            df['Date'] = pd.to_datetime(df['Date'], unit='s')
            df = df.sort_values('Date')
            #df.set_index('Date', inplace=True)
            return df
        return None

    def _on_open(self, symbol, exchange, interval, num_bars):
        """Prepare the on_open callback with the required parameters."""
        def on_open_callback(ws):
            self.logger.info("WebSocket connection established")
            
            def run():
                try:
                    self._send_message(ws, "set_auth_token", ["unauthorized_user_token"])
                    time.sleep(0.2)
                    
                    self._send_message(ws, "chart_create_session", [self.chart_session, ""])
                    time.sleep(0.2)
                    
                    self._send_message(ws, "switch_timezone", [self.chart_session, "Etc/UTC"])
                    time.sleep(0.2)
                    
                    full_symbol = f"{exchange}:{symbol}"
                    symbol_payload = json.dumps({"symbol": full_symbol, "adjustment": "splits"})
                    self._send_message(ws, "resolve_symbol", [self.chart_session, "symbol_1", f"={symbol_payload}"])
                    time.sleep(0.2)
                    
                    self._send_message(ws, "create_series", [self.chart_session, "s1", "s1", "symbol_1", interval, num_bars, ""])
                    time.sleep(0.2)
                    
                    self._send_message(ws, "chart_create_study", [self.chart_session, "st1", "st1", "sds_1", full_symbol, interval, num_bars, ""])
                    
                    self.logger.info(f"Requested {num_bars} {interval} bars for {full_symbol}")
                except Exception as e:
                    self.logger.error(f"Error in initialization sequence: {e}")
            
            thread = threading.Thread(target=run)
            thread.daemon = True
            thread.start()
        
        return on_open_callback

    def _send_message(self, ws, method, params):
        """Send a properly formatted message to TradingView WebSocket."""
        try:
            data = json.dumps({"m": method, "p": params})
            message = f"~m~{len(data)}~m~{data}"
            ws.send(message)
            self.logger.debug(f"Sent: {method} - {params[:50]}{'...' if len(str(params)) > 50 else ''}")
        except Exception as e:
            self.logger.error(f"Error sending message {method}: {e}")

    def _on_message(self, ws, message):
        """Handle incoming WebSocket messages."""
        if message == "~h~":
            self.logger.debug("Received heartbeat, sending pong")
            ws.send("~h~")
            return
        
        if not message.startswith("~m~"):
            return
        
        try:
            msg_parts = message.split("~m~")
            for i in range(0, len(msg_parts)):
                if msg_parts[i] and msg_parts[i].isdigit() and i + 1 < len(msg_parts):
                    content = msg_parts[i+1]
                    if content and content.startswith("{"):
                        try:
                            data = json.loads(content)
                            
                            if isinstance(data, dict) and data.get("m") == "timescale_update":
                                self.logger.debug("Found timescale_update message")
                                payload = data.get("p", [])
                                if len(payload) > 1 and isinstance(payload[1], dict):
                                    for series_id, series_data in payload[1].items():
                                        if "s" in series_data and isinstance(series_data["s"], list):
                                            bars_count = len(series_data["s"])
                                            self.logger.debug(f"Found {bars_count} price bars")
                                            
                                            for bar in series_data["s"]:
                                                if "v" in bar and len(bar["v"]) >= 5:
                                                    ts, o, h, l, c = bar["v"][:5]
                                                    volume = bar["v"][5] if len(bar["v"]) > 5 else None
                                                    self.received_data.append({
                                                        "Date": ts,
                                                        "Open": o,
                                                        "High": h,
                                                        "Low": l,
                                                        "Close": c,
                                                        "Volume": volume
                                                    })
                        except json.JSONDecodeError:
                            self.logger.debug(f"Failed to parse JSON: {content[:50]}...")
        except Exception as e:
            self.logger.error(f"Error processing message: {e}")

    def _on_error(self, ws, error):
        """Handle WebSocket errors."""
        self.error_occurred = True
        self.logger.error(f"WebSocket error: {error}")

    def _on_close(self, ws, close_status_code, close_msg):
        """Handle WebSocket connection close."""
        self.logger.debug(f"WebSocket closed: {close_status_code} - {close_msg}")


# Example usage
def get_data(symbol, exchange="NASDAQ", interval="D", bars=5000, log_level=logging.WARNING):
    """
    Simple function to fetch data from TradingView.
    
    Args:
        symbol: Trading symbol (e.g., "AAPL")
        exchange: Exchange name (default: "NASDAQ")
        interval: Time interval (default: "D" for daily)
        bars: Number of bars to request (default: 5000)
        log_level: Logging level (default: WARNING - minimal output)
    
    Returns:
        pandas.DataFrame: DataFrame with OHLCV data or None if failure
    """
    fetcher = TradingViewDataFetcher(log_level=log_level)
    
    # Alternative URLs to try if the primary fails
    alt_urls = [
        "wss://data.tradingview.com/socket.io/websocket",
        "wss://s.tradingview.com/socket.io/websocket"
    ]
    
    return fetcher.fetch_data(
        symbol=symbol,
        exchange=exchange,
        interval=interval,
        num_bars=bars,
        alt_urls=alt_urls
    )
import re 





In [31]:
import pandas as pd
import time
import hashlib

# Initialize an empty DataFrame to hold all appended data
data = pd.DataFrame()

# Store the last clipboard hash to detect changes
last_clipboard_hash = None

def process_clipboard():
    global data, last_clipboard_hash

    # Read clipboard content as raw text
    clipboard_text = pd.read_clipboard(sep=None).to_csv(index=False, header=False)


    # Hash clipboard content to detect changes
    current_hash = hashlib.md5(clipboard_text.encode('utf-8')).hexdigest()

    if current_hash == last_clipboard_hash:
        # No change in clipboard, skip processing
        return

    # Update last clipboard hash
    last_clipboard_hash = current_hash

    # Load clipboard data into DataFrame
    df = pd.read_clipboard()

    # Split the single column into multiple columns by comma
    df_split = df.iloc[:, 0].str.split(',', expand=True)

    # Remove quotes safely from all cells
    df_split = df_split.applymap(lambda x: x.strip('"') if isinstance(x, str) else x)

    # Select only Date, Open, High, Low, Close, and Volume columns by their positions
    selected_cols = [0, 1, 2, 3, 4, 7]
    df_selected = df_split.iloc[:, selected_cols]

    # Rename columns accordingly
    df_selected.columns = ['Date', 'Open', 'High', 'Low', 'Close', 'Volume']

    # Convert 'Date' to datetime
    df_selected['Date'] = pd.to_datetime(df_selected['Date'])

    # Convert price and volume columns to numeric, coercing errors to NaN
    for col in ['Open', 'High', 'Low', 'Close', 'Volume']:
        df_selected[col] = pd.to_numeric(df_selected[col], errors='coerce')

    # Set 'Date' as index
    df_selected = df_selected.set_index('Date')

    # Append only new rows with datetime index not already in data
    if not data.empty:
        new_data = df_selected.loc[~df_selected.index.isin(data.index)]
        if not new_data.empty:
            data = pd.concat([data, new_data]).sort_index()
    else:
        data = df_selected

    print(f"Appended {len(df_selected)} rows, total rows: {len(data)}")
    print("\nLatest 10 records:")
    print(data.tail(10))
    print("\n---")

# Run every minute
process_clipboard()



Appended 124 rows, total rows: 124

Latest 10 records:
                            Open   High    Low  Close  Volume
Date                                                         
2025-06-26 04:00:00+00:00  10.85  11.38   8.90   9.83       1
2025-06-27 04:00:00+00:00  10.25  12.30   8.28   9.83       2
2025-06-30 04:00:00+00:00  11.06  11.64   9.40  11.10       2
2025-07-01 04:00:00+00:00  22.00  25.62  17.27  19.25       4
2025-07-02 04:00:00+00:00  12.73  17.00  10.40  10.50       5
2025-07-03 04:00:00+00:00   9.90  11.15   8.49   9.60       4
2025-07-07 04:00:00+00:00  22.61  25.24  19.78  21.00       2
2025-07-08 04:00:00+00:00  18.75  20.00  13.82  17.28       3
2025-07-09 04:00:00+00:00  17.55  19.67  15.54  17.35       2
2025-07-10 04:00:00+00:00  14.65  14.65   8.25   8.35       9

---


/var/folders/f_/1bh6mfq97bxdwzwnr49jf8nm0000gn/T/ipykernel_1444/2042354893.py:35: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_split = df_split.applymap(lambda x: x.strip('"') if isinstance(x, str) else x)
/var/folders/f_/1bh6mfq97bxdwzwnr49jf8nm0000gn/T/ipykernel_1444/2042354893.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_selected['Date'] = pd.to_datetime(df_selected['Date'])
/var/folders/f_/1bh6mfq97bxdwzwnr49jf8nm0000gn/T/ipykernel_1444/2042354893.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_gui

In [32]:


#data.index = data.index - pd.Timedelta(hours=2)


In [33]:
from IPython.display import display, HTML

# Get the last 10 rows as HTML
html_table = data.to_html()

# Wrap in a scrollable div
scrollable_html = f'''
<div style="max-height:300px; overflow:auto; border:1px solid #ccc;">
    {html_table}
</div>
'''




In [34]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from filterpy.kalman import KalmanFilter
##___________________________________________________________________________________________________________________________________________________________________________________
#days = 7 # or 7 for a full week
##___________________________________________________________________________________________________________________________________________________________________________________
# --- Data Preparation ---

# Remove weekends and keep only regular trading hours
datafix = data[data.index.weekday < 5]
#datafix = datafix[(datafix.index.hour < 16) & (datafix.index.hour > 9)]

# Filter for only the last N days in the data
last_date = datafix.index.max()
timer = last_date - pd.Timedelta(days=days)
datafix = datafix[datafix.index >= timer]

# Ensure index is DatetimeIndex
datafix.index = pd.to_datetime(datafix.index)

# --- Kalman Filter Price ---

kf = KalmanFilter(dim_x=2, dim_z=1)
kf.x = np.array([datafix['Close'].iloc[0], 0.])  # initial state (price, velocity)
kf.F = np.array([[1., .1], [0., 1.]])            # state transition matrix
kf.H = np.array([[1., 0.]])                      # measurement function
kf.P *= 1000.                                    # covariance matrix
kf.R = 1.                                        # measurement noise
kf.Q = np.eye(2)*0.05                            # process noise

filtered_state_means = []
for price in datafix['Close']:
    kf.predict()
    kf.update(price)
    filtered_state_means.append(kf.x[0])

datafix['Kalman'] = filtered_state_means



In [35]:
##KALMAN FILTER ROC
datafix["Kalman_Roc"] = datafix['Kalman'].pct_change().dropna() * 100  # Convert to percentage change


In [36]:
#MACD
# Calculate MACD and Signal Line
def calculate_macd(df, close_col='Close', fast=12, slow=26, signal=9):
    ema_fast = df[close_col].ewm(span=fast, adjust=False).mean()
    ema_slow = df[close_col].ewm(span=slow, adjust=False).mean()
    macd = ema_fast - ema_slow
    signal_line = macd.ewm(span=signal, adjust=False).mean()
    macd_hist = macd - signal_line
    df['MACD'] = macd
    df['Signal'] = signal_line
    df['MACD_Hist'] = macd_hist
    return df

macdata = calculate_macd(datafix, close_col='Close', fast=12, slow=26, signal=9)

# Plotting MACD
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
import numpy as np

# Split MACD histogram into positive and negative for color distinction
macd_hist = macdata['MACD_Hist']
pos_hist = macd_hist.copy()
neg_hist = macd_hist.copy()
pos_hist[pos_hist < 0] = 0
neg_hist[neg_hist > 0] = 0






In [37]:
import pandas as pd

## adjusting strike price for the contract

import pandas_datareader.data as web
import datetime

start = datetime.datetime(2025, 6, 1)
end = datetime.datetime(2025, 6, 9)

# DGS1MO is the FRED code for the 1-Month Treasury Constant Maturity Rate
rate = web.DataReader('DGS1MO', 'fred', start, end)
rfr = rate.iloc[-1, 0]  # Last available value as a scalar



In [38]:
datafix.tail(10)

,Open,High,Low,Close,Volume,Kalman,Kalman_Roc,MACD,Signal,MACD_Hist
Date,,,,,,,,,,
2025-06-26 04:00:00+00:00,10.85,11.38,8.90,9.83,1,9.782683,-6.896651,-5.418133,-5.342408,-0.075725
2025-06-27 04:00:00+00:00,10.25,12.30,8.28,9.83,2,9.250719,-5.437812,-5.339378,-5.341802,0.002424
2025-06-30 04:00:00+00:00,11.06,11.64,9.40,11.10,2,9.206761,-0.475184,-5.115518,-5.296545,0.181028
2025-07-01 04:00:00+00:00,22.00,25.62,17.27,19.25,4,11.367675,23.470954,-4.231690,-5.083574,0.851885
2025-07-02 04:00:00+00:00,12.73,17.00,10.40,10.50,5,10.792106,-5.063208,-4.189013,-4.904662,0.715649
2025-07-03 04:00:00+00:00,9.90,11.15,8.49,9.60,4,10.125074,-6.180743,-4.179634,-4.759656,0.580023
2025-07-07 04:00:00+00:00,22.61,25.24,19.78,21.00,2,12.641571,24.854111,-3.215252,-4.450776,1.235523
2025-07-08 04:00:00+00:00,18.75,20.00,13.82,17.28,3,13.667821,8.118054,-2.719794,-4.104579,1.384785
2025-07-09 04:00:00+00:00,17.55,19.67,15.54,17.35,2,14.510257,6.163648,-2.295035,-3.742670,1.447635


In [39]:
import pandas as pd

#dateexpiry = "2025-06-20"
expiry = pd.Timestamp(dateexpiry)

# Ensure the index is timezone-naive to avoid subtraction errors
datafix.index = datafix.index.tz_localize(None)

# Calculate days to expiry
datafix['days_to_expiry'] = (expiry - datafix.index).days.astype(float) + 1

#print(datafix['days_to_expiry'].tail(1))

# Continue with your display logic




In [40]:
datafix.tail(10)

,Open,High,Low,Close,Volume,Kalman,Kalman_Roc,MACD,Signal,MACD_Hist,days_to_expiry
Date,,,,,,,,,,,
2025-06-26 04:00:00,10.85,11.38,8.90,9.83,1,9.782683,-6.896651,-5.418133,-5.342408,-0.075725,22.0
2025-06-27 04:00:00,10.25,12.30,8.28,9.83,2,9.250719,-5.437812,-5.339378,-5.341802,0.002424,21.0
2025-06-30 04:00:00,11.06,11.64,9.40,11.10,2,9.206761,-0.475184,-5.115518,-5.296545,0.181028,18.0
2025-07-01 04:00:00,22.00,25.62,17.27,19.25,4,11.367675,23.470954,-4.231690,-5.083574,0.851885,17.0
2025-07-02 04:00:00,12.73,17.00,10.40,10.50,5,10.792106,-5.063208,-4.189013,-4.904662,0.715649,16.0
2025-07-03 04:00:00,9.90,11.15,8.49,9.60,4,10.125074,-6.180743,-4.179634,-4.759656,0.580023,15.0
2025-07-07 04:00:00,22.61,25.24,19.78,21.00,2,12.641571,24.854111,-3.215252,-4.450776,1.235523,11.0
2025-07-08 04:00:00,18.75,20.00,13.82,17.28,3,13.667821,8.118054,-2.719794,-4.104579,1.384785,10.0
2025-07-09 04:00:00,17.55,19.67,15.54,17.35,2,14.510257,6.163648,-2.295035,-3.742670,1.447635,9.0


In [43]:
import pandas as pd

# Example: interval can be 5 (for 5 minutes) or '1D' (for 1 day)
interval = "1D"  # or interval = 5

# Assuming datafix.index is a pandas DatetimeIndex
start = datafix.index[0]
end = datafix.index[-1]

if isinstance(interval, str) and interval.upper().endswith("D"):
    # Interval is in days
    days = int(interval[:-1]) if interval[:-1] else 1
    num_bars = int((end - start) / pd.Timedelta(days=days))
    interval_str = interval  # for get_data
else:
    # Interval is in minutes
    minutes = int(interval)
    num_bars = int((end - start) / pd.Timedelta(minutes=minutes))
    interval_str = str(interval)

# Fetch exactly as many bars as there are rows in datafix
stockdata = get_data(ticker, exchange=exchange, interval=interval_str, bars=num_bars)


In [44]:
datafix.tail(10)

,Open,High,Low,Close,Volume,Kalman,Kalman_Roc,MACD,Signal,MACD_Hist,days_to_expiry
Date,,,,,,,,,,,
2025-06-26 04:00:00,10.85,11.38,8.90,9.83,1,9.782683,-6.896651,-5.418133,-5.342408,-0.075725,22.0
2025-06-27 04:00:00,10.25,12.30,8.28,9.83,2,9.250719,-5.437812,-5.339378,-5.341802,0.002424,21.0
2025-06-30 04:00:00,11.06,11.64,9.40,11.10,2,9.206761,-0.475184,-5.115518,-5.296545,0.181028,18.0
2025-07-01 04:00:00,22.00,25.62,17.27,19.25,4,11.367675,23.470954,-4.231690,-5.083574,0.851885,17.0
2025-07-02 04:00:00,12.73,17.00,10.40,10.50,5,10.792106,-5.063208,-4.189013,-4.904662,0.715649,16.0
2025-07-03 04:00:00,9.90,11.15,8.49,9.60,4,10.125074,-6.180743,-4.179634,-4.759656,0.580023,15.0
2025-07-07 04:00:00,22.61,25.24,19.78,21.00,2,12.641571,24.854111,-3.215252,-4.450776,1.235523,11.0
2025-07-08 04:00:00,18.75,20.00,13.82,17.28,3,13.667821,8.118054,-2.719794,-4.104579,1.384785,10.0
2025-07-09 04:00:00,17.55,19.67,15.54,17.35,2,14.510257,6.163648,-2.295035,-3.742670,1.447635,9.0


In [45]:
import pandas as pd

# Adjust stockdata['Date'] and set as index
stockdata["Date"] = pd.to_datetime(stockdata["Date"]) - pd.Timedelta(hours=0)
stockdata = stockdata.set_index("Date").sort_index()
stockdata["stock price"] = stockdata["Close"]

# Ensure datafix index is datetime and sorted
datafix.index = pd.to_datetime(datafix.index)
datafix = datafix.sort_index()

# Use merge_asof with suffixes to control column names
merged = pd.merge_asof(
    datafix,
    stockdata[["Close"]],
    left_index=True,
    right_index=True,
    direction="nearest",
    suffixes=('_datafix', '_stock')
)

# Rename columns: Close_datafix → Close, Close_stock → stock price
merged = merged.rename(columns={
    'Close_datafix': 'Close',
    'Close_stock': 'stock price'
})

# Assign merged DataFrame back to datafix
datafix = merged

# Show the last 10 rows


In [46]:



# set strike for black scholes 
datafix['strike'] = strike  
datafix['rfr'] = rfr/100
# Define the type of option
datafix["type"] = type
##___________________________________________________________________________________________________________________________________________________________________________________


In [47]:


import numpy as np
from scipy.stats import norm
from scipy.optimize import brentq

def black_scholes_price(S, K, T, r, sigma, option_type):
    d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    if option_type == 'c':
        return S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
    else:
        return K * np.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1)

def implied_volatility_row(row):
    price = row['Close']
    S = row['stock price']
    K = row['strike']
    T = row['days_to_expiry'] / 365.0
    r = row['rfr']
    option_type = row['type']  # 'c' for call, 'p' for put

    # Objective function for root finding
    def objective(sigma):
        return black_scholes_price(S, K, T, r, sigma, option_type) - price

    try:
        return brentq(objective, 1e-6, 5.0)
    except Exception:
        return np.nan

datafix['IV'] = datafix.apply(implied_volatility_row, axis=1)



In [48]:
datafix["IVROC"] = datafix["IV"].pct_change().dropna() * 100  # Convert to percentage change


In [49]:
# Ensure your index is a DatetimeIndex
datafix.index = pd.to_datetime(datafix.index)

# Filter for weekdays (Monday=0, Sunday=6)
is_weekday = datafix.index.dayofweek < 5

# Filter for regular trading hours (9:30 to 16:00)
"""is_regular_hours = (
    (datafix.index.hour > 9) & (datafix.index.hour < 16) |  # 10:00 to 15:59
    ((datafix.index.hour == 9) & (datafix.index.minute >= 30)) |  # 9:30 to 9:59
    ((datafix.index.hour == 16) & (datafix.index.minute == 0))    # exactly 16:00
)
"""
# Combine masks
mask = is_weekday #& is_regular_hours

# Apply mask to keep only regular trading hours
datafix = datafix.loc[mask]


In [50]:

import matplotlib.pyplot as plt

# Reset index and detect the datetime column name
datafix_reset = datafix.reset_index()
datetime_col = datafix_reset.columns[0]  # The first column after reset_index() is always the old index

# For tick labeling, use the datetime column
time_labels = datafix_reset[datetime_col]

# Function to plot with integer index and datetime tick labels
def plot_with_int_index(y_cols, suptitle, kinds=None):
    n = len(y_cols)
    fig, axs = plt.subplots(n, 1, figsize=(12, 3*n), sharex=True)
    if n == 1:
        axs = [axs]
    for i, col in enumerate(y_cols):
        kind = 'line' if not kinds else kinds[i]
        if kind == 'bar':
            axs[i].bar(datafix_reset.index, datafix_reset[col], label=col, alpha=0.7)
        else:
            axs[i].plot(datafix_reset.index, datafix_reset[col], label=col)
        axs[i].set_ylabel(col)
    # Set x-ticks with datetime labels
    step = max(1, len(time_labels) // 10)
    axs[-1].set_xticks(datafix_reset.index[::step])
    axs[-1].set_xticklabels([t.strftime('%Y-%m-%d %H:%M') for t in time_labels[::step]], rotation=45)
    plt.suptitle(suptitle)
    plt.tight_layout()
    plt.show()

# 1. Close, Volume (bar), Kalman
plot_with_int_index(['Close', 'Volume', 'Kalman'], 'Close, Volume, Kalman (No Gaps)', kinds=['line', 'bar', 'line'])

# 2. Close, Kalman_Roc, MACD_Hist (bar)
plot_with_int_index(['Close', 'Kalman_Roc', 'MACD_Hist'], 'Close, Kalman ROC, MACD_Hist (No Gaps)', kinds=['line', 'line', 'line'])
import pandas as pd
from sklearn.preprocessing import StandardScaler
#3. Prices

# Overlay plot: normalized stock price and close
fig, axs = plt.subplots(2, 1, figsize=(12, 6), sharex=True)
axs[0].plot(datafix_reset.index, datafix['stock price'], label='Stock Price ', color='red')
axs[0].set_ylabel('Stock Price')
axs[1].plot(datafix_reset.index, datafix['Close'], label='Close', color='blue', alpha=0.7)
axs[1].set_ylabel('Stock Price & Close')
axs[1].legend()
step = max(1, len(time_labels) // 10)
axs[1].set_xticks(datafix_reset.index[::step])
axs[1].set_xticklabels([t.strftime('%Y-%m-%d %H:%M') for t in time_labels[::step]], rotation=45)
plt.suptitle('Normalized Stock Price and Close (No Gaps)')
plt.tight_layout()
plt.show()


# 4. IV, IVROC
plot_with_int_index(['IV', 'IVROC'], 'IV and IVROC (No Gaps)')



In [51]:
# kalman filter (Garch) the IV
# Pull price data from fidelity csv 
# set thresholds 
# figure out papertrade setup 

In [ ]:
import os

# Assume 'datafix' is your pandas DataFrame
# import pandas as pd
# datafix = pd.DataFrame(...)  # your DataFrame creation

user_input = input("Download datafix.csv to Desktop? (y/n): ").strip().lower()

if user_input == 'y':
    desktop_path = os.path.join(os.path.expanduser("~"), "Desktop")
    file_path = os.path.join(desktop_path, "datafix.csv")
    try:
        datafix.to_csv(file_path, index=True)
        print(f"datafix.csv saved to {file_path}{filenum} (including index)")
    except Exception as e:
        print(f"Failed to save CSV: {e}")
else:
    print("Download canceled.")


datafix.csv saved to /Users/nshaffer/Desktop/datafix.csv (including index)


In [53]:
# get a garch or volatility measure to make trades when IV is low (and increasing?)

In [54]:
datafix.head(10)

,Open,High,Low,Close,Volume,Kalman,Kalman_Roc,MACD,Signal,MACD_Hist,days_to_expiry,stock price,strike,rfr,type,IV,IVROC
Date,,,,,,,,,,,,,,,,,
2025-01-10 05:00:00,25.32,25.44,23.35,24.20,0,24.200000,NaN,0.000000,0.000000,0.000000,189.0,394.74,310,0.0429,p,0.597140,NaN
2025-01-13 05:00:00,24.25,24.37,22.60,22.60,7,22.733663,-6.059243,-0.127635,-0.025527,-0.102108,186.0,403.31,310,0.0429,p,0.602573,0.909739
2025-01-14 05:00:00,20.78,23.75,20.78,23.75,181,23.309758,2.534105,-0.134442,-0.047310,-0.087132,185.0,396.36,310,0.0429,p,0.601402,-0.194281
2025-01-15 05:00:00,21.30,21.30,18.84,18.84,26,20.125958,-13.658659,-0.529924,-0.143833,-0.386091,184.0,428.22,310,0.0429,p,0.612708,1.880019
2025-01-16 05:00:00,19.35,20.50,19.20,20.00,69,19.474572,-3.236546,-0.741200,-0.263306,-0.477893,183.0,413.82,310,0.0429,p,0.597915,-2.414402
2025-01-17 05:00:00,18.09,18.09,16.65,16.65,20,17.399878,-10.653349,-1.165519,-0.443749,-0.721770,182.0,426.50,310,0.0429,p,0.581437,-2.755903
2025-01-21 05:00:00,18.37,18.75,18.35,18.35,17,17.124717,-1.581395,-1.349068,-0.624812,-0.724255,178.0,424.07,310,0.0429,p,0.606146,4.249610
2025-01-22 05:00:00,16.86,16.88,16.85,16.85,14,16.351047,-4.517857,-1.597158,-0.819282,-0.777876,177.0,415.11,310,0.0429,p,0.566437,-6.551062
2025-01-23 05:00:00,17.65,18.25,17.65,17.65,11,16.240931,-0.673446,-1.709512,-0.997328,-0.712184,176.0,412.38,310,0.0429,p,0.572877,1.136937
